Primeiro, importamos a biblioteca "psycopg2", que é utilizada para fazer a conexão do script com a base de dados do PostgreSQL.

In [3]:
import psycopg2
import names
import random as r
import phonenumbers
import pandas as pd

Criada a database no pgAdmin, possível fazer a conexão.

In [4]:
DB_NAME = "heloo_bi"
DB_USER = "postgres"
DB_PASS = "tayrone250595"
DB_HOST = "localhost"

conn = psycopg2.connect(dbname = DB_NAME, user = DB_USER, password = DB_PASS, host = DB_HOST)

O cursor é o objeto utilizado para executar os comandos SQL.

In [5]:
cursor = conn.cursor()

Primeiro, todas as tabelas são criadas, como sugeridas na descrição do teste.

In [40]:
# cursor.execute('''CREATE TABLE medicos(
#     id BIGSERIAL NOT NULL PRIMARY KEY,
#     crm TEXT,
#     nome TEXT,
#     especialidade TEXT,
#     telefone TEXT
#     );''')

# cursor.execute('''CREATE TABLE pacientes(
#     id BIGSERIAL NOT NULL PRIMARY KEY,
#     nome TEXT,
#     telefone TEXT
#     );''')

# cursor.execute('''CREATE TABLE consultas_situacoes(
#     id BIGSERIAL NOT NULL PRIMARY KEY,
#     nome TEXT NOT NULL
#     );''')

# cursor.execute('''CREATE TABLE cidades(
#     id BIGSERIAL NOT NULL PRIMARY KEY,
#     nome TEXT NOT NULL,
#     estado_sigla TEXT NOT NULL
#     );''')

# cursor.execute('''CREATE TABLE consultas(
#     id BIGSERIAL NOT NULL PRIMARY KEY,
#     paciente_id BIGINT NOT NULL REFERENCES pacientes(id), 
#     medico_id BIGINT NOT NULL REFERENCES medicos(id), 
#     data_agendamento TIMESTAMP,
#     situacao_id BIGINT NOT NULL REFERENCES consultas_situacoes(id), 
#     cidade_id BIGINT NOT NULL REFERENCES cidades(id)
#     );''')

# conn.commit()


Para popular a base de dados, vamos gerar mil entradas para a relação "Médicos", usando dados aleatórios.
Primeiro, geramos os nome, usando a biblioteca "names".

In [83]:
names.get_full_name()

'Holly Pratt'

Depois, geramos os números de telefone.

In [7]:
def gerar_telefone():
    telefone = []
    
    # O primeiro número é 9
    telefone.append(9)

    # Os dois próximos são 88 ou 99
    telefone.append(r.randint(8, 9))
    telefone.append(telefone[1])

    # O restante são números de 0 a 9

    for i in range(3, 9):
        telefone.append(r.randint(0, 9))
    

    telefone = int(''.join(map(str, telefone)))
    return telefone

gerar_telefone()

988657806

In [ ]:
Agora, geramos os CRM.

In [8]:
def gerar_crm():
    crm = []

    crm.append('CRM')

    for i in range(0, 4):
        crm.append(r.randint(0, 9))

    crm.append('/PB')   

    crm = (''.join(map(str, crm)))

    return crm

gerar_crm()

'CRM4302/PB'

Para as especialidades, usando a lista com todas as 55 especialidades existentes do Conselhor Federal de Medicina.

In [9]:
with open("especialidades.txt") as file:
    especialidades = file.readlines()
    especialidades = [line.rstrip() for line in especialidades]

especialidades[0:5]

['Acupuntura',
 'Alergia e imunologia',
 'Anestesiologia',
 'Angiologia',
 'Cardiologia']

Agora, podemos definir um dataframe contendo mil instâncias a serem inseridas na relação Médicos.

In [11]:
populacao_medicos = []

for i in range(0, 1000):
    instancia_medico = (gerar_crm(), names.get_full_name(), r.choice(especialidades), gerar_telefone())

    populacao_medicos.append(instancia_medico)

populacao_medicos = pd.DataFrame(populacao_medicos, columns = ['crm', 'nome', 'especialidade', 'telefone'])

In [12]:
cursor.copy_from(populacao_medicos, table = 'medicos', columns = ['crm', 'nome', 'especialidade', 'telefone'])

TypeError: argument 1 must have a .read() method

In [29]:
cursor.close()

conn.close()